In [1]:
#!pip install dash==1.11.0
!pip install dash

In [3]:
import pandas as pd
import numpy as np

housing = pd.read_csv('manhattan_rentals_apr27.csv')
neighborhoods = [x.strip() for x in list(housing['Neighborhood'].value_counts().index)]
neighborhood_options = [{'label': x, 'value': x} for x in neighborhoods]
full_df = pd.read_csv('scored_housing/full_housing.csv')
full_df['Monthly Rental Price'] = full_df['Monthly Rental Price'].astype(float)

## Creating the User Intake Survey on Dash

In [5]:
dislikes = []

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children=["Welcome to ApartmentRanker v.2!"],
           style={
               'textAlign':'center'
           }),
    
    html.Div(children=[
        html.H3(children="Tell us about yourself!",
               style={
                   'textAlign':'center'
               }),
        
        html.Div([
            dcc.Markdown(children=
                        """
                        ApartmentRanker works best if you answer each question honestly as we use this data to match suitable apartments with you. Simply fill out the required questions and see apartments around you today!
                        """)
        ]),
        
        html.Div(children=[
            
            html.H5(children="First, let's talk requirements.",
                    style={'textAlign':'center'}),
            
            html.Div(children=[
                html.Label('Number of bedrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bedrooms-state',
                    options=[
                        {'label': '0 (studio)','value':0.0},
                        {'label': '1 bedroom','value':1.0},
                        {'label': '2 bedrooms','value':2.0},
                        {'label': '3 bedrooms','value':3.0},
                        {'label': '4 bedrooms','value':4.0},
                        {'label': '5 bedrooms','value':5.0}
                    ]),

                html.Label('Number of bathrooms',style={"font-weight":"bold"}),
                dcc.RadioItems(
                    id='bathrooms-state',
                    options=[
                        {'label': '1 bathroom','value':1.0},
                        {'label': '2 bathrooms','value':2.0},
                        {'label': '3 bathrooms','value':3.0},
                        {'label': '4 bathrooms','value':4.0},
                        {'label': '5 bathrooms','value':5.0}
                    ]),

                ],style={'columnCount':2}),
        
            html.Label('Maximum monthly rental price',style={"font-weight":"bold"}),
            dcc.Slider(
                id='price_max-state',
                min=1000,
                max=10000,
                marks={i: '${}'.format(i) for i in range (1000,11000,1000)}
            ),
            
            html.Label('Preferred neighborhoods',style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='neighborhood-state',
                options=neighborhood_options,
                multi=False)
        
        ]),
        
        html.Div(children=[
            
            html.H5(children="One last thing, tell us about your priorities!",
                    style={'textAlign':'center'}),
            
            html.Label("1st priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority1-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("2nd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority2-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),    
                 
            html.Label("3rd priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority3-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),   
                 
            html.Label("4th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority4-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),
                 
            html.Label("5th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority5-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),  
                 
            html.Label("6th priority",style={"font-weight":"bold"}),
            dcc.Dropdown(
                id='priority6-state',
                options=[
                    {'label':'Proximity to grocery stores','value':"grocer"},
                    {'label':'Proximity to nightlife','value':"nightlife"},
                    {'label':'Noise-free','value':"noise"},
                    {'label':'Proximity to variety of restaurants/cafes','value':"food"},
                    {'label':'Proximity to a subway','value':"subway"},
                    {'label':'Proximity to greenery','value':"trees"},
                ]),                 
            ]),
        
        html.Button(id='submit-button-state', children='Submit')
        
        ],
            style=
            {"margin":"0 auto",
             'max-width':"650px",
            }),
    
        html.Div(id='initial_recc')
    ])

@app.callback(
    Output('initial_recc', 'children'),
    [Input('submit-button-state','n_clicks')],
    [State('bedrooms-state','value'),
    State('bathrooms-state','value'), 
    State('price_max-state','value'),
    State('neighborhood-state','value'),
    State('priority1-state','value'),
    State('priority2-state','value'),
    State('priority3-state','value'),
    State('priority4-state','value'),
    State('priority5-state','value'),
    State('priority6-state','value')])
def initial_rec(n_clicks,beds,bath,price,neighbor,p1,p2,p3,p4,p5,p6):
    #preferences is a list with the ranked features of
    # grocers_normalized
    # nightlife_normalized
    # noise_normalized
    # restaurants_normalized
    # subways_normalized
    # trees_normalized
    
    counter=1

    tree_pref = 0
    subway_pref = 0
    grocer_pref = 0
    noise_pref = 0
    restaurants_pref = 0
    nightlife_pref = 0

    p1="nightlife"
    p2="food"
    p3="noise"
    p4="grocer"
    p5="subway"
    p6="trees"

    for p in [p1,p2,p3,p4,p5,p6]:
        if p=="trees":
            tree_pref=counter
            counter+=1
            continue
        if p=="subway":
            subway_pref=counter
            counter+=1
            continue
        if p=="grocer":
            grocer_pref=counter
            counter+=1
            continue
        if p=="noise":
            noise_pref=counter
            counter+=1
            continue
        if p=="food":
            restaurants_pref=counter
            counter+=1
            continue
        if p=="nightlife":
            nightlife_pref=counter
            counter+=1
            continue
            
    preferences = np.array([grocer_pref, nightlife_pref, noise_pref, restaurants_pref, subway_pref, tree_pref])
    
    new_df = full_df[(full_df['Bedrooms'] == beds) & (full_df['Bathrooms'] == bath) & (full_df['Monthly Rental Price'] <= price) & (full_df['Neighborhood'] == neighbor)]
    new_df["User Score"] = 0.0
    
    scores = []
    for i in range(new_df.shape[0]):
        score = preferences * np.array(new_df.iloc[i, 19:25] + 10)
        scores.append(sum(score))
    new_df['User Score'] = scores
    new_df = new_df.sort_values(by = ['User Score'], ascending = False)
    
    counter_1 = 0
    
    while True:
        potential = new_df.iloc[counter_1, :]
        if potential not in dislikes:
            break
        else:
            counter_1 = counter_1 + 1
    result = potential
    
    return ("We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) +
     " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms.")

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Apr/2020 19:14:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 19:14:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2020 19:14:17] "GET / HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "//anaconda3/lib/python3.7/site-packages/dash/dash.py", line 967, in dispatch
    response.set_data(func(*args, output

127.0.0.1 - - [28/Apr/2020 19:14:19] "POST /_dash-update-component HTTP/1.1" 500 -
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:234: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:240: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

127.0.0.1 - - [28/Apr/2020 19:15:32] "POST /_dash-update-component HTTP/1.1" 200 -


In [63]:
dislikes = []
def initial_rec(beds,bath,price,neighbor,p1,p2,p3,p4,p5,p6):
    #preferences is a list with the ranked features of
    # grocers_normalized
    # nightlife_normalized
    # noise_normalized
    # restaurants_normalized
    # subways_normalized
    # trees_normalized
    
    counter=1

    tree_pref = 0
    subway_pref = 0
    grocer_pref = 0
    noise_pref = 0
    restaurants_pref = 0
    nightlife_pref = 0

    p1="nightlife"
    p2="food"
    p3="noise"
    p4="grocer"
    p5="subway"
    p6="trees"

    for p in [p1,p2,p3,p4,p5,p6]:
        if p=="trees":
            tree_pref=counter
            counter+=1
            continue
        if p=="subway":
            subway_pref=counter
            counter+=1
            continue
        if p=="grocer":
            grocer_pref=counter
            counter+=1
            continue
        if p=="noise":
            noise_pref=counter
            counter+=1
            continue
        if p=="food":
            restaurants_pref=counter
            counter+=1
            continue
        if p=="nightlife":
            nightlife_pref=counter
            counter+=1
            continue
            
    preferences = np.array([grocer_pref, nightlife_pref, noise_pref, restaurants_pref, subway_pref, tree_pref])
    
    new_df = full_df[(full_df['Bedrooms'] == beds) & (full_df['Bathrooms'] == bath) & (full_df['Monthly Rental Price'] <= price) & (full_df['Neighborhood'] == neighbor)]
    new_df["User Score"] = 0.0
    
    scores = []
    for i in range(new_df.shape[0]):
        score = preferences * np.array(new_df.iloc[i, 19:25] + 10)
        scores.append(sum(score))
    new_df['User Score'] = scores
    new_df = new_df.sort_values(by = ['User Score'], ascending = False)
    
    counter_1 = 0
    
    while True:
        potential = new_df.iloc[counter_1, :]
        if potential not in dislikes:
            break
        else:
            counter_1 = counter_1 + 1
    result = potential
    
    return ("We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) +
     " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms.")
    

In [64]:
initial_rec(1.0,1.0,10000.00,"Battery Park City",p1,p2,p3,p4,p5,p6)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



'We recommend: 70 Battery Place #607 in Battery Park City. This listing is $3950 per month. It has 1 bedrooms and 1 bathrooms.'

In [62]:
result = initial_rec(1.0,1.0,10000.00,"Battery Park City",p1,p2,p3,p4,p5,p6)
result

We recommend: 70 Battery Place #607 in Battery Park City. This listing is $3950 per month. It has 1 bedrooms and 1 bathrooms.


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Unnamed: 0                                         223
Unnamed: 0.1                                       223
Unnamed: 0.1.1                                     223
Listing ID                                     2951585
Location                         70 Battery Place #607
Lat_Long                  ['40.70722719,-74.01757146']
Monthly Rental Price                              3950
Bedrooms                                             1
Bathrooms                                            1
Square Footage                                     686
Latitude                                       40.7072
Longitude                                     -74.0176
Type                                         Apartment
num_grocers                                          4
num_nightlife                                        1
num_noise                                           46
num_restaurants                                     12
num_subways                                          3
num_trees 

In [60]:
print("We recommend: " + result['Location'] + " in " + result['Neighborhood'] + ". This listing is $" + str(int(result['Monthly Rental Price'])) +
     " per month. It has " + str(int(result['Bedrooms'])) + " bedrooms and " + str(int(result['Bathrooms'])) + " bathrooms.")



We recommend: 70 Battery Place #607 in Battery Park City. This listing is $3950 per month. It has 1 bedrooms and 1 bathrooms.


In [46]:
preferences = np.array([tree_pref, subway_pref, grocer_pref, noise_pref, restaurants_pref, nightlife_pref])

hello = full_df[(full_df['Bedrooms'] == 1.0) & (full_df['Bathrooms'] == 1.0) & (full_df['Monthly Rental Price'] <= 10000.0) & (full_df['Neighborhood'] == "Battery Park City")]
hello['User Score'] = 0.0

scores = []
for i in range(hello.shape[0]):
    score = preferences * np.array(hello.iloc[i, 19:25] + 10)
    scores.append(sum(score))
hello['User Score'] = scores
hello = hello.sort_values(by = ['User Score'], ascending = False)
    
    

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [10]:
counter=1

tree_pref = 0
subway_pref = 0
grocer_pref = 0
noise_pref = 0
restaurants_pref = 0
nightlife_pref = 0

p1="nightlife"
p2="food"
p3="noise"
p4="grocer"
p5="subway"
p6="trees"

for p in [p1,p2,p3,p4,p5,p6]:
    if p=="trees":
        tree_pref=counter
        counter+=1
        continue
    if p=="subway":
        subway_pref=counter
        counter+=1
        continue
    if p=="grocer":
        grocer_pref=counter
        counter+=1
        continue
    if p=="noise":
        noise_pref=counter
        counter+=1
        continue
    if p=="food":
        restaurants_pref=counter
        counter+=1
        continue
    if p=="nightlife":
        nightlife_pref=counter
        counter+=1
        continue

In [12]:
np.array(tree_pref, subway_pref, grocer_pref, noise_pref, restaurants_pref, nightlife_pref)

(6, 5, 4, 3, 2, 1)